In [ ]:
%load_ext autoreload
%autoreload 2

# Rescale features

This notebooks computes the mean and std for various features.
We will use these values to rescale the data before training.

In [ ]:
import dask
import dask_jobqueue
import dask.dataframe as dd
import dask.distributed
import os
import numpy as np
import pathlib
import pandas as pd

## Boot cluster



In [ ]:
cluster = dask_jobqueue.SLURMCluster(
    env_extra=['source ~/.bash_profile','conda activate smc01'],
    name='smc01-dask',
)

In [ ]:
cluster.scale(jobs=6)

In [ ]:
client = dask.distributed.Client(cluster)

In [ ]:
client

In [ ]:
DATA_DIR = pathlib.Path(os.getenv('DATA_DIR'))
DATASET_DIR = DATA_DIR / 'interpolated/2021-12-20-gdps-metar/'

In [ ]:
dataset_path = pathlib.Path(DATASET_DIR)

In [ ]:
train_files = [f for f in dataset_path.glob('*.parquet') if int(f.name[:4]) in [2019,2020]]

In [ ]:
train_files[:10]

In [ ]:
df = dd.read_parquet(train_files)

In [ ]:
df.head()

## Means and stds

In [ ]:
mean = df.mean().compute()

In [ ]:
std = df.std().compute()

In [ ]:
mean_dict = mean

In [ ]:
for k, v in mean_dict.items():
    print(f'"{k}": {v:.4e},')

In [ ]:
std_dict = dict(std)

In [ ]:
for k, v in std_dict.items():
    print(f'"{k}": {v:.4e},')

## Other fields

### Elevation

In [ ]:
elevation = df['elevation'].compute()

In [ ]:
elevation.plot.hist()

In [ ]:
elevation_by_station = df.groupby('station').agg({'elevation': 'first'}).compute()

In [ ]:
elevation_by_station[elevation_by_station['elevation'] < 0.0]

In [ ]:
elevation_by_station.min()

In [ ]:
np.log(elevation + 44 + 1e-6)

In [ ]:
np.log(elevation+ 1e-6).plot.hist()

In [ ]:
np.log(elevation + 1e-6).mean()

In [ ]:
np.log(elevation + 1e-6).std()

### GDPS precipitation rate (Prate)

In [ ]:
gdps_prate = df['gdps_prate'].compute()

In [ ]:
gdps_prate.plot.hist()

In [ ]:
(gdps_prate**(1./3.)).plot.hist()

## 10m wind speed (si)

In [ ]:
obs_10si = df['obs_10si'].compute()

In [ ]:
obs_10si.plot.hist(range=(0,10))

In [ ]:
obs_10si.mean()

In [ ]:
obs_10si.std()

In [ ]:
obs_10si.max()

In [ ]:
df[df['obs_10si'] > 400].compute()

In [ ]:
obs_10si.quantile(q=0.01)

### Relative Humidity (Obs and GDPS)

In [ ]:
obs_2r = df['obs_2r'].compute()

In [ ]:
obs_2r.plot.hist()

In [ ]:
obs_2r.max()

In [ ]:
obs_2r.quantile(q=0.999)

In [ ]:
df['gdps_2r'].compute().plot.hist()

### OBS Mean Sea Level Pressure

In [ ]:
obs_prmsl = df['obs_prmsl'].compute()

In [ ]:
obs_prmsl.plot.hist()

In [ ]:
obs_prmsl.min()

In [ ]:
obs_prmsl.max()

### GDPS Albedo

In [ ]:
gdps_alb = df['gdps_al'].compute()

In [ ]:
gdps_alb.plot.hist()

### GDPS Wind direction

In [ ]:
gdps_wdir = df['gdps_10wdir'].compute()

In [ ]:
gdps_wdir.plot.hist()